In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import re

In [3]:
df = pd.read_csv('DATA COLLECTION/dipresive_normal_text_data.csv')

In [4]:
df

,text,labels
0,love them. RT @passport_cutty: Do you like Gem...,Normal
1,I have a habbit of nibbling on my lip or the i...,Normal
2,Happy Wellness Wednesday!! Be proud of all the...,Normal
3,Wake up and Bake up! Thank God for another 24!...,Normal
4,Lesson learned: suicide jokes don't hit the sa...,Dipresive
...,...,...
28849,This is a pretty good idea. Would like to see...,Dipresive
28850,Hey guys. Please block and report this account...,Dipresive
28851,Send some love to @SheridanLoveXXX as she goes...,Normal
28852,Can I ask where you get your statics? I'm in m...,Dipresive


In [4]:
def clean_tweet(tweet):
    ''' 
    Utility function to clean tweet text by removing links, special characters 
    using simple regex statements. 
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
# predict single value

In [5]:
df['text'] = df['text'].apply(clean_tweet)

In [6]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['labels'], random_state=0)
X_train = X_train.apply(lambda x: np.str_(x))
X_test = X_test.apply(lambda x: np.str_(x))

In [7]:
# Train and evaluate the model
vect = CountVectorizer().fit(X_train)
X_train_vectorized = vect.transform(X_train)
X_test_vectorized = vect.transform(X_test)

In [8]:
clfrNB = MultinomialNB(alpha=0.1)
clfrNB.fit(X_train_vectorized, y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [9]:
pre = clfrNB.predict(X_test_vectorized)

In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pre)

0.9675630718048239

In [12]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pre))

              precision    recall  f1-score   support

   Dipresive       0.94      1.00      0.97      3644
      Normal       1.00      0.94      0.97      3570

    accuracy                           0.97      7214
   macro avg       0.97      0.97      0.97      7214
weighted avg       0.97      0.97      0.97      7214



In [13]:
def predictsenti(text):
    text = clean_tweet(text)
    pre = clfrNB.predict(vect.transform([text]))
    if pre[0] == 'Dipresive':
        return 'Dipresive'
    else:
        return 'Normal'

In [16]:
predictsenti('I am not happy with my life')

'Normal'

In [24]:
from eli5 import show_prediction
show_prediction(clfrNB, 'I am not happy with my life', vec=vect, show_feature_values=True)